In [1]:
!pip install requests beautifulsoup4 pandas openpyxl nltk textblob


In [2]:
import pandas as pd

# Load the Excel file (already uploaded to /mnt/data)
input_path = "/content/Input.xlsx"
df = pd.read_excel(input_path)

# Show first few entries
df.head()


,URL_ID,URL
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...
4,Netclan20241021,https://insights.blackcoffer.com/development-o...


In [3]:
import os
import requests
from bs4 import BeautifulSoup

# Create folder to store articles
os.makedirs("articles", exist_ok=True)

def extract_article_text(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        title = soup.find('h1')
        content = soup.find('div', class_='td-post-content')  # Blackcoffer articles use this class

        title_text = title.get_text(strip=True) if title else ''
        content_text = content.get_text(separator=' ', strip=True) if content else ''

        return title_text + "\n\n" + content_text
    except Exception as e:
        print(f"❌ Error fetching {url}: {e}")
        return ''

# Loop through each URL and save .txt file
for _, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    article = extract_article_text(url)

    if article.strip():  # Save only if not empty
        with open(f"articles/{url_id}.txt", "w", encoding="utf-8") as f:
            f.write(article)
    else:
        print(f"⚠️ Empty article for URL_ID {url_id}")


❌ Error fetching https://insights.blackcoffer.com/an-etl-solution-for-currency-data-to-google-big-query/: HTTPSConnectionPool(host='insights.blackcoffer.com', port=443): Read timed out. (read timeout=10)
⚠️ Empty article for URL_ID Netclan20241095


In [4]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob

nltk.download('punkt')

# Define helper functions
def count_syllables(word):
    word = word.lower()
    vowels = "aeiou"
    count = 0
    if word[0] in vowels:
        count += 1
    for i in range(1, len(word)):
        if word[i] in vowels and word[i - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

def is_complex(word):
    return count_syllables(word) >= 3

def count_pronouns(text):
    return len(re.findall(r'\b(I|we|my|ours|us)\b', text, flags=re.I))

# Example positive/negative word lists (use full dictionaries if provided)
positive_words = set(["good", "great", "positive", "fortunate", "excellent", "correct", "superior", "benefit"])
negative_words = set(["bad", "worst", "negative", "unfortunate", "wrong", "inferior", "harm"])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
!pip install chardet


In [6]:
import chardet

def detect_encoding(filepath):
    with open(filepath, 'rb') as f:
        raw_data = f.read()
    result = chardet.detect(raw_data)
    return result['encoding']

def load_words(filepath):
    encoding = detect_encoding(filepath)
    with open(filepath, 'r', encoding=encoding) as f:
        return set(line.strip() for line in f if line.strip() and not line.startswith(';'))

# Load files
positive_words = load_words('/content/positive-words.txt')
negative_words = load_words('/content/negative-words.txt')


In [7]:
import pandas as pd

# Load Excel file
df = pd.read_excel('/content/Input.xlsx')  # or the path where you've uploaded it

# Show column names to verify
print("Columns in Excel:", df.columns.tolist())


Columns in Excel: ['URL_ID', 'URL']


In [8]:
def analyze_sentiment(text, pos_words, neg_words):
    words = text.lower().split()
    pos_count = sum(1 for word in words if word in pos_words)
    neg_count = sum(1 for word in words if word in neg_words)

    if pos_count > neg_count:
        return 'Positive'
    elif neg_count > pos_count:
        return 'Negative'
    else:
        return 'Neutral'


In [9]:
TEXT_COLUMN = 'Content'


In [10]:
TEXT_COLUMN = 'YourCorrectColumnName'  # e.g., 'Review', 'Text', etc.


In [11]:
print(df.columns.tolist())


['URL_ID', 'URL']


In [3]:
!pip install --upgrade --force-reinstall newspaper3k


  Using cached newspaper3k-0.2.8-py3-none-any.whl.metadata (11 kB)
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached pillow-11.3.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached cssselect-1.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached lxml-6.0.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached tldextract-5.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached feedfinder2-0.0.4-py3-none-any.whl
  Using cached jieba3k-0.35.1-py3-none-any.whl
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached tinysegmenter-0.3-py3-no

In [2]:
!pip install lxml[html_clean]

In [1]:
import newspaper
print("Newspaper version:", newspaper.__version__)


Newspaper version: 0.2.8


In [2]:
from newspaper import Article
import pandas as pd

# Load input Excel file
df = pd.read_excel('/content/Input.xlsx')  # replace with your actual file path

# Function to extract article text
def extract_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except:
        return ""

# Create 'Content' column by applying extraction
df['Content'] = df['URL'].apply(extract_article)

# Save to new file
df.to_excel('output_with_content.xlsx', index=False)


In [4]:
# Load word lists
def load_words(filepath):
    with open(filepath, 'r', encoding='ISO-8859-1') as file:
        words = [line.strip() for line in file if line.strip() and not line.startswith(';')]
    return set(words)

positive_words = load_words('/content/positive-words.txt')
negative_words = load_words('/content/negative-words.txt')

# Function to analyze sentiment
def analyze_sentiment(text, pos_words, neg_words):
    text = text.lower()
    words = text.split()
    pos_count = sum(1 for word in words if word in pos_words)
    neg_count = sum(1 for word in words if word in neg_words)

    if pos_count > neg_count:
        return "Positive"
    elif neg_count > pos_count:
        return "Negative"
    else:
        return "Neutral"

# Load the file with content
df = pd.read_excel('output_with_content.xlsx')

# Apply sentiment analysis
df['Sentiment'] = df['Content'].apply(lambda x: analyze_sentiment(str(x), positive_words, negative_words))

# Save final output
df.to_excel('final_output_with_sentiment.xlsx', index=False)

In [5]:
from google.colab import files
files.download('final_output_with_sentiment.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
!pip install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.8 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=fc1cffacd5499ff72d30a4d330810444477238429a2e840b93c260681ce6d3b1
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=03e828dc2121f02e58a15f96002064d5c307e1e499ad77531d280bda33b3b221
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1ec

In [6]:
!pip install newspaper3k
!pip install lxml_html_clean

In [7]:
from newspaper import Article

In [8]:
import pandas as pd
from newspaper import Article

df = pd.read_excel('/content/Input.xlsx')

def extract_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except:
        return ""

df['Content'] = df['URL'].apply(extract_article)
df.to_excel('output_with_content.xlsx', index=False)


In [9]:
from google.colab import files
files.download("output_with_content.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>